In [1]:
import escher
from escher import Builder
import cobra
from time import sleep

In [2]:
escher.rc['never_ask_before_quit'] = True

In [3]:
# builder

In [4]:

builder = Builder(
    model_json='iAB_RBC_283.json'
)
builder
    

Builder(never_ask_before_quit=True)

In [5]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [6]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [7]:
builder

Builder(never_ask_before_quit=True)

In [8]:
# Load an Escher map
builder.map_name = 'iJO1366.Central metabolism'

In [9]:

# Load a COBRA model
builder.model_name = 'e_coli_core'

In [10]:

# Find any reactions that are in the map and not in the model, and turn them red
builder.highlight_missing = True

In [11]:

builder.model = cobra.io.load_json_model('iJO1366.json')

In [12]:
solution = builder.model.optimize()
builder.reaction_data = solution.fluxes

# Add some data for metabolites
builder.metabolite_data = solution.shadow_prices


# Add some data for metabolites
builder.metabolite_data = solution.shadow_prices

builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]

builder.reaction_scale_preset = 'GaBuRd'


# Make all the arrows three times as thick
builder.reaction_scale = [
    {k: v * 3 if k == 'size' else v for k, v in x.items()}
    for x in builder.reaction_scale
]

In [13]:
builder = Builder(
    height=600,
    map_name=None,
    model_name='e_coli_core',
    map_json='https://escher.github.io/1-0-0/5/maps/Escherichia%20coli/e_coli_core.Core%20metabolism.json',
)
builder

Builder(height=600, never_ask_before_quit=True)

In [14]:
model = cobra.io.load_json_model('e_coli_core.json')

In [15]:
reverse = False
step = 0.1
timestep = 0.1
duration = 15 # seconds
lim = [0, 0.5]
val = lim[-1]
for _ in range(int(duration / timestep)):
    model.reactions.EX_o2_e.lower_bound = -val
    solution = model.optimize()
    builder.reaction_data = solution.fluxes
    if val <= lim[0]:
        reverse = True
    if val >= lim[1]:
        reverse = False
    if reverse:
        val += step
    else:
        val -= step
    sleep(timestep)

C:\Users\eswar\anaconda3\envs\ML\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [16]:
builder.save_html('example_map.html')